## Sampling and Analysis - Dublin
This file is a replica of point_density, but considers sample surfaces in the Dublin dataset.  

This notebook performs the central sampling and analysis of the paper. For each sample surface (defined by a set of xyz coordinates on the plane), the notebook generates the desired number of sample squares, collects the points in that square, generates the desired statistics, and aggregates over all the sample surfaces.  Most of the functions utilized are from the point_density_functions.py file.  

Additional analyses:
* Density at different wall heights
* Missing points by scan angle, missing points on vertical vs. horizontal surfaces
* Figure 12 analysis - mean orthogonal offset by flight pass, which generates the cross-pass error.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from laspy.file import File
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

import sys

sys.path.insert(0,'..') # So we can import point_density_functions from parent directory
from point_density_functions import *
%load_ext autoreload
%matplotlib notebook

In [ ]:
# Dublin tile
file_dir = '../../Data/dublin_sample/'
# filename = 'bank_of_ireland.las'
filename = 'dublin_horizontal.las'
pt_files_vertical = ['las_points_bank_of_ireland.lz']
                    


# Corresponds to LAS 1.2 Point Data Record Format 1
columns_dublin_pt_cloud = [
    'X',
    'Y',
    'Z',
    'intensity',
    'return_number_byte',
    'classification_byte',
    'scan_angle',
    'user_data',
    'pt_src_id',
    'gps_time']

In [ ]:
# This works
# for file in filenames:
#     create_df_hd5(file_dir,file,columns_dublin_pt_cloud)

In [ ]:
# Load the lz file
# bank_df = pd.read_hdf(file_dir+'las_points_bank_of_ireland.lz')
# horiz_df = pd.read_hdf(file_dir+'las_points_dublin_horizontal.lz')
# horiz_df = pd.read_hdf(file_dir+'las_points_dublin_horizontal.lz')

### Supporting Functions

In [ ]:
def print_out(SampleFlightList,pt_density_laefer_list,feet_from_point):
    # Laefer
    sd_laefer_total = np.mean([ss.flight_list_laefer[0].sd_dist for ss in SampleFlightList])
    sd_laefer_sample = np.mean([ss.flight_list_laefer[1].sd_dist for ss in SampleFlightList])
    avg_flight_paths_laefer = np.mean([len(flight.flight_list_laefer) for flight in SampleFlightList])-2
    sd_flight_paths_laefer = np.std([len(flight.flight_list_laefer) for flight in SampleFlightList])
    phis_laefer_total = [ss.phi_laefer_total for ss in SampleFlightList]
    phis_laefer_sample = [ss.phi_laefer_sample for ss in SampleFlightList]

    print("2019 scan (Horizontal, {} samples): \n".format(len(SampleFlightList))+"*"*30)
    print("Avg points per square: {:2.2f} points".format((4 * feet_from_point**2)*np.mean(pt_density_laefer_list)))
    print("Avg density: {:2.4f} pts/m^2 (SD: {:2.4f})".format(np.mean(pt_density_laefer_list),np.std(pt_density_laefer_list)))
    print("Avg number of flight paths per square: {:2.4f} (SD: {:2.4f})".format(avg_flight_paths_laefer,sd_flight_paths_laefer))
    print("\nphi_total: {:2.4f} (SD: {:2.4f})".format(np.mean(phis_laefer_total),np.std(phis_laefer_total)))
    print("phi_sample: {:2.4f} (SD: {:2.4f})".format(np.mean(phis_laefer_sample),np.std(phis_laefer_sample)))
    print("Total point dist from plane, SD: {:2.4f} m".format(sd_laefer_total))
    print("Avg flight point dist from plane, SD: {:2.4f} m".format(sd_laefer_total/np.mean(phis_laefer_total)))

## Sampling squares and Statistics

### Horizontal Density
The section below iterates through the sample surfaces, generating sample squares, generating density/accuracy statistics for each, and gathering the results.

In [ ]:
# Aggregators - collect sample squares across multiple sample areas
pt_density_list = []
avg_height_diff = []
sd_height = []
SampleFlightList = []

In [ ]:
pt_files_horizontal = ['las_points_leinster_house_lot_1.lz', \
                       'las_points_leinster_house_lot_2.lz', \
                       'las_points_stephensgreen_walking.lz', \
                       'las_points_city_hall_square.lz'
                      ]

In [ ]:
### Horizontal - City Hall Square
pt1 = np.array([315483.8333,234007.297119,6.139])
pt2 = np.array([315481.253052,234021.491943,6.034])
u_length = np.linalg.norm(pt2-pt1) # ~14.1m
v_length = 14
pt3 = np.array([3.15497605e+05, 2.34009799e+05, 5.87147628e+00]) # Backed into from previous 2D v calculation
uv_inv,w,unit_u,unit_v = rectangle(pt1[:2],pt2[:2],u_length,v_length)

rectangle_points = pd.read_hdf(file_dir+pt_files_horizontal[3])
rectangle_points = rectangle_points[(rectangle_points['z_scaled']<6.2)& \
                                                  (rectangle_points['z_scaled']>5.7)]

rectangle_points['flight_id'] = rectangle_points['pt_src_id']
# Norm from any 3 points on plane
# a = np.array([315485.316040,234016.809082,6.011])
# b = np.array([315483.8333,234007.297119,6.139])
# c = np.array([315493.062,234019.477051,5.84700])
# norm = np.cross(c - b,(a-b))

# bottom_vec = (np.cross(norm,(pt2 - pt1)))/np.linalg.norm(np.cross(norm,(pt2 - pt1)))
# bottom_right_pt = pt1 - bottom_vec*v_length
# bottom_right_pt
print(u_length*v_length)

In [ ]:
# Create center_points
feet_from_point = 0.5
border = []
border.append(1.05*feet_from_point/u_length)
border.append(1.05*feet_from_point/v_length)
center_points = center_point_sample(2800,pt1,pt2,pt3,u_length=u_length,v_length=v_length,border=border)
mean_z = rectangle_points['z_scaled'].mean()
pts_thrown_out = 0

for center_point in center_points:
    square_points = in_horizontal_square(rectangle_points,center_point[:2],feet_from_point)
    
    flight_count = len(square_points['flight_id'].unique())
   
    # If z_max > 6.2, for any point in square skip it
    if square_points['z_scaled'].max()<6.2:
        # Statistics!
                
        # Point density
        num_points = square_points.shape[0]
        pt_density_list.append(num_points / (4 * feet_from_point**2))        
        
        # Flight path specifics
        flight_list = create_flight_list(square_points)
        # Create SampleSquare from all flight passes
        ss = SampleSquare(flight_list, x = center_point[0], \
                          y=center_point[1],feet_from_point=feet_from_point)
        # Collect SampleSquares
        SampleFlightList.append(ss)
        
    else:
        pts_thrown_out +=1

In [ ]:
print_out(SampleFlightList,pt_density_list,feet_from_point)

In [ ]:
cw_df = pd.DataFrame([flight.error_decomp_laefer for flight in SampleFlightList],columns=['laefer_C','laefer_W','laefer_rmse'])

cw_df.describe()

In [ ]:
### Horizontal - stephensgreen walking
pt1 = np.array([315925.706055,233508.467041,11.846])
pt2 = np.array([315920.5,233521.358887,11.669])
u_length = np.linalg.norm(pt2-pt1) # ~14m
v_length = 5
pt3 = np.array([3.15930342e+05, 2.33510339e+05, 1.18615851e+01]) # Backed into from previous 2D v calculation
uv_inv,w,unit_u,unit_v = rectangle(pt1[:2],pt2[:2],u_length,v_length)

rectangle_points = pd.read_hdf(file_dir+pt_files_horizontal[2])

rectangle_points['flight_id'] = rectangle_points['pt_src_id']
# Norm from any 3 points on plane
# a = pt1
# b = np.array([315929.901978,233511.337891,11.855])
# c = np.array([315924.854004,233518.982910,11.796])
# norm = np.cross(c - b,(a-b))

# bottom_vec = (np.cross(norm,(pt2 - pt1)))/np.linalg.norm(np.cross(norm,(pt2 - pt1)))
# bottom_right_pt = pt1 - bottom_vec*v_length
# bottom_right_pt
print(u_length*v_length)

In [ ]:
# Create center_points
feet_from_point = 0.5
border = []
border.append(1.05*feet_from_point/u_length)
border.append(1.05*feet_from_point/v_length)
center_points = center_point_sample(2500,pt1,pt2,pt3,u_length=u_length,v_length=v_length,border=border)
mean_z = rectangle_points['z_scaled'].mean()
pts_thrown_out = 0

for center_point in center_points:
    square_points = in_horizontal_square(rectangle_points,center_point[:2],feet_from_point)
    
    flight_count = len(square_points['flight_id'].unique())
   
    # If z_max > 10, <6 for any dataset, skip it
    if square_points['z_scaled'].max()<mean_z+1:
        # Statistics!
                
        # Point density
        num_points = square_points.shape[0]
        pt_density_list.append(num_points / (4 * feet_from_point**2))        
        
        # Flight path specifics
        flight_list = create_flight_list(square_points)
        # Create SampleSquare from all flight passes
        ss = SampleSquare(flight_list, x = center_point[0], \
                          y=center_point[1],feet_from_point=feet_from_point)
        # Collect SampleSquares
        SampleFlightList.append(ss)
        
    else:
        pts_thrown_out +=1

In [ ]:
print_out(SampleFlightList,pt_density_list,feet_from_point)

In [ ]:
cw_df = pd.DataFrame([flight.error_decomp_laefer for flight in SampleFlightList],columns=['laefer_C','laefer_W','laefer_rmse'])

cw_df.describe()

In [ ]:
### Horizontal - leinster house lot 1
pt1 = np.array([316315.836914,233643.625977,10.56])
pt2 = np.array([316286.843994,233655.563965,10.523])
u_length = np.linalg.norm(pt2-pt1) # ~31m
v_length = 4.9
pt3 = np.array([3.16317702e+05, 2.33648155e+05, 1.07051393e+01]) # Backed into from previous 2D v calculation
uv_inv,w,unit_u,unit_v = rectangle(pt1[:2],pt2[:2],u_length,v_length)

rectangle_points = pd.read_hdf(file_dir+pt_files_horizontal[0])
# rectangle_points = rectangle_points[(rectangle_points['z_scaled']<6.2)& \
#                                                   (rectangle_points['z_scaled']>5.7)]

rectangle_points['flight_id'] = rectangle_points['pt_src_id']
# # # Norm from any 3 points on plane
# a = pt1
# b = np.array([316290.827881,233658.487061,10.519])
# c = np.array([316312.126953,233650.197998,10.664])
# norm = np.cross(c - b,(a-b))

# bottom_vec = (np.cross(norm,(pt2 - pt1)))/np.linalg.norm(np.cross(norm,(pt2 - pt1)))
# bottom_right_pt = pt1 + bottom_vec*v_length
# print(bottom_right_pt)
print(u_length*v_length)

In [ ]:
# Create center_points
feet_from_point = 0.5
border = []
border.append(1.05*feet_from_point/u_length)
border.append(1.05*feet_from_point/v_length)
center_points = center_point_sample(2500,pt1,pt2,pt3,u_length=u_length,v_length=v_length,border=border)
mean_z = rectangle_points['z_scaled'].mean()
pts_thrown_out = 0

for center_point in center_points:
    square_points = in_horizontal_square(rectangle_points,center_point[:2],feet_from_point)
    
    flight_count = len(square_points['flight_id'].unique())
   
    # If z_max > 10, <6 for any dataset, skip it
    if square_points['z_scaled'].max()<mean_z+1:
        # Statistics!
                
        # Point density
        num_points = square_points.shape[0]
        pt_density_list.append(num_points / (4 * feet_from_point**2))        
        
        # Flight path specifics
        flight_list = create_flight_list(square_points)
        # Create SampleSquare from all flight passes
        ss = SampleSquare(flight_list, x = center_point[0], \
                          y=center_point[1],feet_from_point=feet_from_point)
        # Collect SampleSquares
        SampleFlightList.append(ss)
        
    else:
        pts_thrown_out +=1

In [ ]:
print_out(SampleFlightList,pt_density_list,feet_from_point)

In [ ]:
cw_df = pd.DataFrame([flight.error_decomp_laefer for flight in SampleFlightList],columns=['laefer_C','laefer_W','laefer_rmse'])

cw_df.describe()

In [ ]:
### Horizontal - leinster house lot 2
pt1 = np.array([316307.085938,233679.488037,10.607])
pt2 = np.array([316293.058105,233682.521973,10.478])
u_length = np.linalg.norm(pt2-pt1) # ~14m
v_length = 4.5
pt3 = np.array([3.16307932e+05, 2.33683397e+05, 1.05349566e+01]) # Backed into from previous 2D v calculation
uv_inv,w,unit_u,unit_v = rectangle(pt1[:2],pt2[:2],u_length,v_length)

rectangle_points = pd.read_hdf(file_dir+pt_files_horizontal[1])
rectangle_points = rectangle_points[(rectangle_points['z_scaled']<10.7)& \
                                                  (rectangle_points['z_scaled']>10.4)]

rectangle_points['flight_id'] = rectangle_points['pt_src_id']
# # Norm from any 3 points on plane
# a = pt1
# b = np.array([316294.368896,233686.305908,10.505])
# c = np.array([316307.704102,233681.699951,10.566])
# norm = np.cross(c - b,(a-b))

# bottom_vec = (np.cross(norm,(pt2 - pt1)))/np.linalg.norm(np.cross(norm,(pt2 - pt1)))
# bottom_right_pt = pt1 + bottom_vec*v_length
# print(bottom_right_pt)
print(u_length*v_length)

In [ ]:
# Create center_points
feet_from_point = 0.5
border = []
border.append(1.05*feet_from_point/u_length)
border.append(1.05*feet_from_point/v_length)
center_points = center_point_sample(2500,pt1,pt2,pt3,u_length=u_length,v_length=v_length,border=border)
mean_z = rectangle_points['z_scaled'].mean()
pts_thrown_out = 0

for center_point in center_points:
    square_points = in_horizontal_square(rectangle_points,center_point[:2],feet_from_point)
    
    flight_count = len(square_points['flight_id'].unique())
   
    # If z_max > 10, <6 for any dataset, skip it
    if square_points['z_scaled'].max()<mean_z+1:
        # Statistics!
                
        # Point density
        num_points = square_points.shape[0]
        pt_density_list.append(num_points / (4 * feet_from_point**2))        
        
        # Flight path specifics
        flight_list = create_flight_list(square_points)
        # Create SampleSquare from all flight passes
        ss = SampleSquare(flight_list, x = center_point[0], \
                          y=center_point[1],feet_from_point=feet_from_point)
        # Collect SampleSquares
        SampleFlightList.append(ss)
        
    else:
        pts_thrown_out +=1

In [ ]:
print_out(SampleFlightList,pt_density_list,feet_from_point)

In [ ]:
cw_df = pd.DataFrame([flight.error_decomp_laefer for flight in SampleFlightList],columns=['laefer_C','laefer_W','laefer_rmse'])

cw_df.describe()

## Flight pass height distribution

In [ ]:
# Collect all heights by flight id
from collections import defaultdict

flight_id_dict = defaultdict(list)

for sample_num in range(len(SampleFlightList)):

    dd = dict([(fp.flight_id,fp.h) for fp in SampleFlightList[sample_num].flight_list_laefer[2:]])
    for key in dd.keys():
        flight_id_dict[key].append(dd[key])

h_dist = []
for key in flight_id_dict.keys():
    mean_h = np.mean([abs(v) for v in flight_id_dict[key]])
    h_dist.append(1000*mean_h)
    print("{:2}: {:2.4f}".format(key,mean_h))

# Plot distribution of mean abs(heights)
plt.hist(h_dist,density=True)
plt.title("Dublin Horizontal Surfaces",fontsize=12)
plt.xlabel("Mean absolute height (mm)",fontsize=12)
plt.ylabel("Distribution of flight passes",fontsize=12)

## Vertical density
The section below iterates through the sample surfaces, generating sample squares, generating density/accuracy statistics for each, and gathering the results.

In [ ]:
'''flight_id_mapping = {
	"F_150326_122941":7,
	"F_150326_123430":8,
	"F_150326_123922": 9,
	"F_150326_124415":10,
	"F_150326_154909":32,
	"F_150326_155238":33,
	"F_ 150326_155529":34,
	"F_150326_155833":35
}'''

In [ ]:
# Calculating the bottom right point
# a = (np.cross(norm,(top_left_pt - bottom_left_pt)))/np.linalg.norm(np.cross(norm,(top_left_pt - bottom_left_pt)))
# bottom_right_pt = bottom_left_pt + a*v_length

In [ ]:
def vert_density(square_points, 
                 middle_pt, 
                 right_pt, 
                 left_pt, 
                 bottom_left_pt,
                 u_length,
                 v_length,
                 feet_from_pt,
                 SampleFlightList=[],
                 pt_density_list=[],
                 sd_wall=[],
                 wall_face=None
                 ): 
    
    # Calculate norm_vector from 3 points, to define plane and extract the wall face
    norm = np.cross(middle_pt - right_pt,(left_pt-right_pt))
    norm = norm / np.linalg.norm(norm)

    # Extract the wall face, above 6.5m and below 34m to avoid roof and ground points
    wall_face = grab_wall_face(square_points,norm, middle_pt,6.5,34,5e-1)

    # Fit a plane, create norm_vector, calculate dist_from_plane
    norm_vector,_,wall_face,_ = plane_fit(wall_face) 
    
    # Sample points in wall
    #Note 0.28 border_v comes from: (3.5/2) (half-meter feet_from_point) / v_length (with some buffer)
    feet_from_pt_v = 1
    feet_from_pt_u = 1
    border = [0,0]
    border[0] = 1.05*(feet_from_pt_u / u_length)
    border[1] = 1.05*(feet_from_pt_v / v_length)

    center_points = center_point_sample(5000,
                        bottom_left_pt,top_left_pt,bottom_right_pt,
                        u_length=u_length,v_length=v_length,border=border)
    
    # Main Loop through the sample points
    pts_thrown_out = 0

    for center_point in center_points:
        square_points,pt_density = in_vertical_square(wall_face,
                                                    norm_vector,
                                                    center_point,
                                                    feet_from_pt_v,
                                                    feet_from_pt_u)
        flight_count = len(square_points['flight_id'].unique())
        
        # Statistics!

        # Point density
        num_points = square_points.shape[0]
        pt_density_list.append(pt_density)        
        

        # Fit a plane
        if square_points.shape[0] > 0:
            norm_vector,_,square_points,_ = plane_fit(square_points)
            flight_list = create_flight_list(square_points)
        else:
            flight_list=None

        # Flight path specifics
        ss = SampleSquare(flight_list, x = center_point[0], \
                          y=center_point[1],z=center_point[2],feet_from_point=[feet_from_pt_v,feet_from_pt_u])
        SampleFlightList.append(ss)    
    
    # Print outs
    print("Pts in dataset: {}".format(wall_face.shape[0]))
    
    print("Vertical face point density over {:d} samples".format(len(pt_density_list)))
    print("*"*30)
    print("Avg density: {:2.4f} (SD: {:2.4f})".format(np.mean(pt_density_list),np.std(pt_density_list)))
    
    return SampleFlightList, wall_face, pt_density_list, sd_wall

In [ ]:
# Aggregators
pt_density_list= []
sd_wall=[]
SampleFlightList = []

In [ ]:
# Main wall on Bank of Ireland
middle_pt = np.array([316566.945068,234635.687012,17.288])

# Extract from all files, the points within feet_from_pt in the xy-plane of the middle wall point
square_points_vertical = grab_points(pt_files_vertical,file_dir,middle_pt[0],middle_pt[1],4.15)
square_points_vertical['flight_id'] = square_points_vertical['pt_src_id']

In [ ]:
# calculate the norm
middle_pt = np.array([316566.945068,234635.687012,17.288])
right_pt = np.array([316572.521973,234635.230957,29.448999])
left_pt = np.array([316563.1521, 234635.970215, 22.895])
bottom_left_pt= np.array([316563.1521, 234635.970215, 8.895])
top_left_pt = np.array([316563.1521, 234635.970215, 33.895])
bottom_right_pt = np.array([316571.4272615415,234635.3285753328,8.895])
feet_from_pt = 4.15
u_length = 25
v_length = 8.3

In [ ]:
print("RMSEV: ",(.0166/.0291)-1)
print("C: ",.0072/.025-1)
print("W: ",.01457/.0149-1)

In [ ]:
SampleFlightList, wall_face, pt_density_list, sd_wall = \
vert_density(square_points_vertical, 
                 middle_pt, 
                 right_pt, 
                 left_pt, 
                 bottom_left_pt, 
                 u_length,
                 v_length,
                 feet_from_pt,
                 SampleFlightList,    
                 pt_density_list,
                 sd_wall
                 )

In [ ]:
flight_ids = wall_face['pt_src_id'].unique()
wall_face['pts_bins'] = pd.cut((wall_face['z_scaled']-3.5), \
                                      bins=range(0,36,3), \
                                      labels=range(3,36,3))

for f in flight_ids:
    flight_pts = wall_face[wall_face['pt_src_id']==f]
    # Num of points
    print("\n\nDensity for flight id {}:".format(f),flight_pts.groupby('pts_bins')['X'].count()/(3*v_length))

#### Second wall

In [ ]:
# Main wall on Bank of Ireland
middle_pt = np.array([316543.00,234675.172852,18.693])

# Extract from all files, the points within feet_from_pt in the xy-plane of the middle wall point
square_points_vertical = grab_points(pt_files_vertical,file_dir,middle_pt[0],middle_pt[1],3)
square_points_vertical['flight_id'] = square_points_vertical['pt_src_id']

In [ ]:
# calculate the norm
middle_pt = np.array([316543.00,234675.172852,18.693]) ##
right_pt = np.array([316542.769043,234672.121094,19.367]) ##
left_pt = np.array([316543.187988,234677.369141,26.181]) ##
bottom_left_pt= np.array([316543.198975,234677.737793,6]) ##
top_left_pt = np.array([316543.198975,234677.737793,28.5]) ##
bottom_right_pt = np.array([3.16542722e+05, 2.34671757e+05, 6.00000000e+00])
feet_from_pt = 3 ##
u_length = 25
v_length = 6

In [ ]:
norm_vector,_,_,_ = plane_fit(square_points_vertical)
wall_face = grab_wall_face(square_points_vertical,norm_vector, middle_pt,6.5,34,5e-1)

In [ ]:
SampleFlightList, wall_face, pt_density_list, sd_wall = \
vert_density(square_points_vertical, 
                 middle_pt, 
                 right_pt, 
                 left_pt, 
                 bottom_left_pt, 
                 u_length,
                 v_length,
                 feet_from_pt,
                 SampleFlightList,    
                 pt_density_list,
                 sd_wall
                 )

In [ ]:
flight_ids = wall_face['pt_src_id'].unique()
wall_face['pts_bins'] = pd.cut((wall_face['z_scaled']-3.5), \
                                      bins=range(0,36,3), \
                                      labels=range(3,36,3))

for f in flight_ids:
    flight_pts = wall_face[wall_face['pt_src_id']==f]
    # Num of points
    print("\n\nDensity for flight id {}:".format(f),flight_pts.groupby('pts_bins')['X'].count()/(3*(8.3+6)))

In [ ]:
square_points_vertical['pt_src_id'].value_counts()

In [ ]:
avg_flight_paths_laefer = []
for flight in SampleFlightList:
    try:
        avg_flight_paths_laefer.append(len(flight.flight_list_laefer)-2)
    except AttributeError:
        avg_flight_paths_laefer.append(0)

print("Avg number of flight paths per square: {:2.2f}".format(np.mean(avg_flight_paths_laefer)))

In [ ]:
cw = []
for flight in SampleFlightList:
    try:
        cw.append(flight.error_decomp_laefer)
    except AttributeError:
        print("Missing")
#     cw_df = pd.DataFrame(cw,columns=['laefer_C','laefer_W','laefer_rmse'])
    cw_df = pd.DataFrame(cw,columns=['laefer_C','laefer_W','laefer_rmse'])

(cw_df).describe()

## Flight pass height distribution

In [ ]:
# Collect all heights by flight id
from collections import defaultdict

flight_id_dict = defaultdict(list)

for sample_num in range(len(SampleFlightList)):

    dd = dict([(fp.flight_id,fp.h) for fp in SampleFlightList[sample_num].flight_list_laefer[2:]])
    for key in dd.keys():
        flight_id_dict[key].append(dd[key])

h_dist = []
for key in flight_id_dict.keys():
    mean_h = np.mean([abs(v) for v in flight_id_dict[key]])
    h_dist.append(1000*mean_h)
    print("{:2}: {:2.4f}".format(key,mean_h))

# Plot distribution of mean abs(heights)
plt.hist(h_dist)
plt.title("Dublin Horizontal Surfaces",fontsize=12)
plt.xlabel("Mean absolute height (mm)",fontsize=12)
plt.ylabel("Distribution of flight passes",fontsize=12)

In [ ]:
flight_id_dict[7]

## Where on the  wall are the points from each flight pass? 

In [ ]:
plt.figure(figsize=[20,22])
for i,fid in enumerate(wall_face['flight_id'].unique()):
    plt.subplot(4,3,i+1)
    pts = wall_face[wall_face['flight_id']==fid]
    hist = pd.cut(wall_face[wall_face['flight_id']==fid]['z_scaled'],bins=range(0,120,5),labels=range(5,120,5))
#     plt.hist(hist,orientation='horizontal')
    plt.plot(pts['x_scaled'],pts['z_scaled'],'x')
    plt.yticks(np.arange(0,36,3))
    plt.ylabel("Wall height (m)")
#     plt.xlabel("Point number (not spatial)")
    plt.xlabel("X")
    plt.title(fid)

In [ ]:
wall_face['flight_id'].unique()

## Heatmap by flight pass

In [ ]:
# square_points_list, SampleFlightList, wall_face_list, pt_density_list, sd_wall_list
h_dict = {
 7:[],
 8:[],
 9:[],
 32:[],
 33:[],
 34:[],
}
# Collect h for each flight pass in each sample square
for sample_num in range(10000):
    h_list = [fp.h for fp in SampleFlightList[sample_num].flight_list_laefer[2:]]
    num_pt_list = [fp.num_points for fp in SampleFlightList[sample_num].flight_list_laefer[2:]]
    flight_ids = [fp.flight_id for fp in SampleFlightList[sample_num].flight_list_laefer[2:]]
    total_pts = SampleFlightList[sample_num].flight_list_laefer[0].num_points
    for a in h_dict.keys():
        if a in flight_ids:
            ix = flight_ids.index(a)
            h_dict[a].append(h_list[ix])
        else:
            h_dict[a].append(np.nan)

In [ ]:
# Mean height for each flight pass
print("F_Id\t Mean abs height\n","*"*23)
[print("{}: {:6.3f}".format(k,np.nanmean([abs(h) for h in h_dict[k]]))) for k in h_dict.keys()]

In [ ]:
# Matrix of mean height differences across flight passes (would expect larger diffs for different directions)
def create_h_matrix(h_dict):
    h_matrix = np.zeros([len(h_dict),len(h_dict)])
    for i,key1 in enumerate(h_dict.keys()):
        for j,key2 in enumerate(h_dict.keys()):
            h_matrix[i,j] = np.nanmean(abs(np.array(h_dict[key1]) - np.array(h_dict[key2])))
    return h_matrix

def h_heatmap(h_matrix,h_dict,fontsize=15,label='Flight ID'):
    # Heatmap of the matrix
    plt.figure(figsize=[10,10])
    plt.imshow(h_matrix, cmap='YlOrRd',vmin=0,vmax=0.6)
    plt.xticks(np.arange(0,h_matrix.shape[0]),h_dict.keys(),fontsize=fontsize)
    plt.yticks(np.arange(0,h_matrix.shape[0]),h_dict.keys(),fontsize=fontsize)
    plt.ylabel(label,fontsize=fontsize)
    plt.xlabel(label,fontsize=fontsize)
    plt.ylim(h_matrix.shape[0]-0.5,-0.5)
    plt.title("Mean Absolute h difference",fontsize=fontsize)
    plt.colorbar()
    plt.show()
    
h_matrix = create_h_matrix(h_dict)
h_heatmap(h_matrix,h_dict)  

## Missing points by scan angle

Count the missing scan points based on scan angle between consecutive points.  Goal is to 1) compare the % of missing points for horizontal vs vertical surfaces, and 2) compare the % of missing points at different wall heights

In [ ]:
wall_face_list[0].to_pickle("../../Data/parking_lot/wall_points_laefer.pkl")

In [ ]:
def pull_first_scan_gap(wall_face_laefer):
    # Separate return num, only keep the first returns, add scan_gap, sort
    wall_face_laefer['num_returns'] = np.floor(wall_face_laefer['flag_byte']/16).astype(int)
    wall_face_laefer['return_num'] = wall_face_laefer['flag_byte']%16
    first_return_wall = wall_face_laefer[wall_face_laefer['return_num']==1]
    first_return_wall.sort_values(by=['gps_time'],inplace=True)
    first_return_wall.reset_index(inplace=True)
    first_return_wall.loc[1:,'scan_gap'] = [first_return_wall.loc[i+1,'scan_angle'] - first_return_wall.loc[i,'scan_angle'] for i in range(first_return_wall.shape[0]-1)]
    first_return_wall.loc[0,'scan_gap'] = 0
    return first_return_wall

# Wall
wall_face_laefer = wall_face_list[0]
first_return_wall = pull_first_scan_gap(wall_face_laefer)
# Rectangle
rectangle_face_laefer = pd.read_pickle(file_dir+"rectangle_points_laefer.pkl")
first_return_rectangle = pull_first_scan_gap(rectangle_face_laefer)

In [ ]:
# Scan gap of -4,-5,-6 is normal, -10,-11,-15 are a missed point
w = pd.DataFrame(first_return_rectangle['scan_gap'].value_counts())
w.head(20)

In [ ]:
# Scan gap of -4,-5,-6 is normal, -10,-11,-15 are a missed point
w = pd.DataFrame(first_return_wall[first_return_wall['flight_id']=='180819']['scan_gap'].value_counts())
w.head(20)

In [ ]:
# Split the wall height into bins, compare % missing points at different heights

first_return_wall['pts_bins'] = pd.cut((first_return_wall['z_scaled']-9)/3.28084, \
                                       bins=range(0,40,3),labels=range(3,40,3))

first_return_wall['missed_point'] = np.zeros(first_return_wall.shape[0])
first_return_wall['good_point'] = np.zeros(first_return_wall.shape[0])
for index, row in first_return_wall.iterrows():
    if (row['scan_gap'] >-17) & (row['scan_gap']< -6):
        first_return_wall.loc[index,'missed_point']=1
    if (row['scan_gap'] <-1) & (row['scan_gap'] > -7):
        first_return_wall.loc[index,'good_point']=1

In [ ]:
first_return_wall[first_return_wall['z_scaled']<18]

In [ ]:
# a = first_return_wall[first_return_wall['flight_id']=='181004'].groupby('pts_bins').mean()
a = first_return_wall.groupby('pts_bins').mean()

a['miss_pct'] = a['missed_point']/a['good_point']
a['miss_pct'][:-2].index

In [ ]:
a['miss_pct']

In [ ]:
plt.scatter(list(a['miss_pct'][:-2]),list(a['miss_pct'][:-2].index))

## Intensity up and down the wall

In [ ]:
wall_face_laefer = wall_face_list[0]
flight_ids = ['181004','180819']
wall_face_laefer['pts_bins'] = pd.cut((wall_face_laefer['z_scaled']-9)/3.28084,bins=range(0,40,3),labels=range(3,40,3))

In [ ]:
wall_face_laefer['intensity'].mean()

In [ ]:
flight_pts = wall_face_laefer[wall_face_laefer['flight_id']==flight_ids[1]]
print("shape",flight_pts.shape)
flight_pts.groupby('pts_bins')['intensity'].mean()

In [ ]:
# Intensity on the parking lot horizontal surface
rectangle_points_laefer = pd.read_pickle(file_dir+"rectangle_points_laefer.pkl")
rectangle_points_laefer[rectangle_points_laefer['flight_id']=='181004']['intensity'].mean()

## Density up and down the wall

In [ ]:
wall_face_laefer = wall_face_list[0]
plt.figure(figsize=[20,22])
for i,fid in enumerate(wall_face_laefer['flight_id'].unique()):
    plt.subplot(4,3,i+1)
    pts = wall_face_laefer[wall_face_laefer['flight_id']==fid]
    hist = pd.cut(wall_face_laefer[wall_face_laefer['flight_id']==fid]['z_scaled'],bins=range(0,120,5),labels=range(5,120,5))
#     plt.hist(hist,orientation='horizontal')
    plt.plot(range(pts.shape[0]),pts['z_scaled']/3.28084,'x')
    plt.yticks(np.arange(0,36,3))
    plt.ylabel("Wall height (m)")
    plt.xlabel("Point number (not spatial)")
    plt.title(fid)

In [ ]:
flight_ids = ['181004','180819','164445','180632']
pts_fid = wall_face_laefer[wall_face_laefer['flight_id']==flight_ids[1]]['z_scaled'].copy()
pts_fid = pts_fid/3.28084
pts_bins = pd.cut(pts_fid,bins=range(15,36,3),labels=range(18,36,3))
pts_density = pts_bins.value_counts()/(3*(6.899845795386761/3.28084))
pts_density.sort_index(inplace=True)
plt.plot(pts_density,range(18,36,3),'-o')
plt.xlabel("Vertical Density (pts/m^2)",fontsize=15)
plt.xticks(fontsize=15)
plt.ylabel("Height on wall (m)",fontsize=15)
plt.yticks(fontsize=15)


In [ ]:
pts_fid = wall_face_laefer['z_scaled'].copy()
pts_fid = pts_fid/3.28084
pts_bins = pd.cut(pts_fid,bins=[0,16,32],labels=('low','high'))
pts_density = pts_bins.value_counts()/(1*(6.899845795386761/3.28084))
pts_density.sort_index(inplace=True)
plt.bar(['low','high'],pts_density)
plt.xlabel("Vertical Density (pts/m^2)",fontsize=15)
plt.xticks(fontsize=15)
plt.ylabel("Height on wall (m)",fontsize=15)
plt.yticks(fontsize=15)

In [ ]:
pts_fid = wall_face_laefer['z_scaled'].copy()
pts_fid = pts_fid/3.28084
pts_bins = pd.cut(pts_fid,bins=range(9,36,3),labels=range(12,36,3))
pts_density = pts_bins.value_counts()/(2*(14.899845795386761/3.28084))
pts_density.sort_index(inplace=True)
plt.plot(pts_density,range(12,36,3),'-o')
plt.xlabel("Vertical Density (pts/m^2)",fontsize=15)
plt.xticks(fontsize=15)
plt.ylabel("Height on wall (m)",fontsize=15)
plt.yticks(fontsize=15)

## Accuracy up the wall

In [ ]:
pts_fid = [sf.z for sf in SampleFlightList]
pts_fid = [pt/3.28084 for pt in pts_fid]
pts_bins = pd.cut(pts_fid,bins=range(8,36,2),labels=range(10,36,2))

# Create acc_df Dataframe of accuracy and height bins for a specific flight id
# acc_df = pd.DataFrame([ss.flight_list_laefer[0].sd_dist for ss in SampleFlightList],columns=['total_rmse'])
acc_df = pd.DataFrame(pts_bins,columns=['bin'])

acc_df['z'] = pts_fid
# plt.plot(acc_df[acc_df['bin']==30]['z'])

# Flight id specific accuracy
fid = '181004' # Farther away 
# fid = '180819' # About 100m away
acc_list = []
for j in SampleFlightList:
    dd = {j.flight_list_laefer[i].flight_id:i for i in range(len(j.flight_list_laefer))}
    ix = dd[fid]
    acc_list.append(j.flight_list_laefer[ix].sd_dist/3.28084)
len(acc_list)

acc_df['fid_rmse'] = acc_list

In [ ]:
plt.barh(np.arange(12-3,33-3,2),acc_df.groupby('bin')['fid_rmse'].mean().iloc[1:-1])
plt.ylabel("Wall Height (m)")
plt.xlabel("Single Flight RMSE (m)")
plt.title("Flight ID: "+str(fid))
print(acc_df['bin'].value_counts())

In [ ]:
plt.barh(np.arange(12-3,33-3,2),acc_df.groupby('bin')['fid_rmse'].mean().iloc[1:-1])
plt.ylabel("Wall Height (m)")
plt.xlabel("Single Flight RMSE (m)")
plt.title("Flight ID: "+str(fid))
print(acc_df['bin'].value_counts())

## What causes C and W?
What's the distribution of h's that generate C for vertical surfaces?  Outlier, or consistent misalignment?

In [ ]:
[print("{}:\t{:2.4f}\t{:2} points\t{:2.4f}".format(flight_ids[i],h_list[i]**2*num_pt_list[i],num_pt_list[i],h_list[i])) for i in range(len(h_list))]

In [ ]:
direction = {
 '164239':'n-s  ',
 '164445':'n-s  ',
 '164640':'n-s  ',
 '172753':'e-w  ',
 '172928':'e-w  ',
 '173110':'e-w  ',
 '180632':'sw-ne',
 '180819':'sw-ne',
 '181004':'sw-ne',
 '200212':'se-nw',
 '200600':'se-nw',
 '200742':'se-nw',
 '200938':'se-nw' 
}
dir_list = ['n-s  ','e-w  ','sw-ne','se-nw']

In [ ]:
# square_points_list, SampleFlightList, wall_face_list, pt_density_list, sd_wall_list
h_dict = {
 '164239':[],
 '164445':[],
 '164640':[],
 '172753':[],
 '172928':[],
 '173110':[],
 '180632':[],
 '180819':[],
 '181004':[],
 '200212':[],
 '200600':[],
 '200742':[],
 '200938':[] 
}
# Collect h for each flight pass in each sample square
for sample_num in range(2500):
    h_list = [fp.h for fp in SampleFlightList[sample_num].flight_list_laefer[2:]]
#     num_pt_list = [fp.num_points for fp in SampleFlightList[sample_num].flight_list_laefer[2:]]
    flight_ids = [fp.flight_id for fp in SampleFlightList[sample_num].flight_list_laefer[2:]]
#     total_pts = SampleFlightList[sample_num].flight_list_laefer[0].num_points
    for a in h_dict.keys():
        if a in flight_ids:
            ix = flight_ids.index(a)
            h_dict[a].append(h_list[ix])
        else:
            h_dict[a].append(np.nan)

In [ ]:
# Mean height for each flight pass
print("F_Id\t Mean abs height\n","*"*23)
[print("{}: {:6.3f}".format(k,np.nanmean([abs(h) for h in h_dict[k]]))) for k in h_dict.keys()]

In [ ]:
from collections import defaultdict

h_dir_dict = defaultdict(list)
for ss in range(len(h_dict['164239'])):
    for di in dir_list:
        direction_h_sum = 0
        direction_count = 0

        for fp in h_dict.keys():
            if direction[fp] == di:
                direction_h_sum += h_dict[fp][ss]
                direction_count +=1    
        h_dir_dict[di].append(direction_h_sum/direction_count)
dir_list = ['n-s  ','e-w  ','sw-ne','se-nw']
# Mean height for each direction
print("Direct\tMean abs height\n","*"*22)
[print("{}: {:6.3f}".format(k,np.nanmean([abs(h) for h in h_dir_dict[k]]))) for k in h_dir_dict.keys()]

In [ ]:
# Matrix of mean height differences across flight passes (would expect larger diffs for different directions)
def create_h_matrix(h_dict):
    h_matrix = np.zeros([len(h_dict),len(h_dict)])
    for i,key1 in enumerate(h_dict.keys()):
        for j,key2 in enumerate(h_dict.keys()):
            h_matrix[i,j] = np.nanmean(abs(np.array(h_dict[key1]) - np.array(h_dict[key2])))
    return h_matrix

def h_heatmap(h_matrix,h_dict,fontsize=15,label='Flight ID'):
    # Heatmap of the matrix
    plt.figure(figsize=[10,10])
    plt.imshow(h_matrix, cmap='YlOrRd')
    plt.xticks(np.arange(0,h_matrix.shape[0]),h_dict.keys(),rotation=90,fontsize=fontsize)
    plt.yticks(np.arange(0,h_matrix.shape[0]),h_dict.keys(),fontsize=fontsize)
    plt.ylabel(label,fontsize=fontsize)
    plt.xlabel(label,fontsize=fontsize)
    plt.ylim(h_matrix.shape[0]-0.5,-0.5)
    plt.title("Mean Absolute h difference",fontsize=fontsize)
    plt.colorbar()
    plt.show()
    
h_matrix = create_h_matrix(h_dict)
h_heatmap(h_matrix,h_dict)  
# h_matrix = create_h_matrix(h_dir_dict)
# h_heatmap(h_matrix,h_dir_dict)

In [ ]:
h_matrix = create_h_matrix(h_dir_dict)
h_heatmap(h_matrix,h_dir_dict,label="Flight Direction")